In [20]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100

import geopandas as gpd

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt


from os import listdir
import unicodedata


from paths import *
from scripts.io import read_sheets
from scripts import io


from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

from IPython.display import IFrame


import yaml
config = yaml.load(open('../configs/config.yaml', 'r'), Loader=yaml.FullLoader)


# Brasil.IO

In [21]:
from scripts import io

brio, brio_raw = io.load_brasilIO()

print("Br.IO table, Done!")

Br.IO table, Done!


In [22]:
brio

,city_ibge_code,city,confirmed,deaths,date,state
442916,1200013.0,Acrelândia,270,6,2020-07-29,AC
442917,1200054.0,Assis Brasil,311,6,2020-07-29,AC
442918,1200104.0,Brasiléia,763,11,2020-07-29,AC
442919,1200138.0,Bujari,318,5,2020-07-29,AC
442920,1200179.0,Capixaba,205,7,2020-07-29,AC
...,...,...,...,...,...,...
469312,4323754.0,Vitória das Missões,2,0,2020-08-02,RS
469313,4323770.0,Westfália,59,0,2020-08-02,RS
469314,4323804.0,Xangri-lá,62,3,2020-08-02,RS
469315,4300059.0,Água Santa,69,0,2020-08-02,RS


## WCota
 - [github](https://github.com/wcota/covid19br)

In [23]:
# #data https://github.com/wcota/covid19br
# dd = io.load_wcota()


## MYTABLE

In [24]:
from scripts.io import read_sheets
from scripts import manipulation
config = yaml.load(open('../configs/config.yaml', 'r'), Loader=yaml.FullLoader)

# io.update_ms_data()

df = pd.read_csv("../data/ministerio_da_saude/last_data_ms_covid19.csv")


df = df.drop_duplicates(subset = ['regiao', 'estado', 'data', 'casosAcumulado', 'obitosAcumulado', 'last_update'])


df_states = manipulation.manipule_mytable(df,config['ms_table'])


In [25]:
vale = read_sheets('covid19_vale_do_paraiba_e_litoral_norte').replace('',0)
print("Vale table, Done!")

Vale table, Done!


In [26]:
br = io.load_total_table()
br = manipulation.create_all_country_total_data(br, vars = ['confirmed','deaths','recovered','active'])

In [27]:
br.head()

,date,countrycode,countryname,population,confirmed,new_confirmed,deaths,new_deaths,recovered,new_recovered,active,new_active
29608,2020-01-20,AF,Afghanistan,32225560.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39508,2020-01-20,TG,Togo,7538000.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33208,2020-01-20,DE,Germany,83149300.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39448,2020-01-20,TH,Thailand,66484730.0,2,2.0,0.0,0.0,0.0,0.0,2.0,2.0
33268,2020-01-20,GH,Ghana,30280811.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### GRAPHS

[Local - Indicator Model](file:///mnt/0BF8481B1CDBF12D/Jlab/gabinete_sv/covid19/images/storage/br_indicator_model.html)

[Local - Indicator Final](file:///mnt/0BF8481B1CDBF12D/Jlab/gabinete_sv/covid19/images/storage/br_indicator_final.html)

[GS-Indicator_Final](https://storage.googleapis.com/sv-covid19/brasil/br_indicator_final.html)

In [28]:
from scripts import vis_html

config = yaml.load(open('../configs/config.yaml', 'r'), Loader=yaml.FullLoader)

vis_html.create_cards(df_states,vale, br ,config['embed_html'])


#     io.to_storage(bucket='sv-covid19',
#               bucket_folder='site/full',
#               file_name=files[model],
#               path_to_file=path+files[model])
    
# print("Embed link uploaded")

Embed html uploaded!


## Cumulativo

In [29]:
from scripts import vis_graphs

themes = yaml.load(open('../themes/custom_colorscales.yaml', 'r'), Loader=yaml.FullLoader)
themes = themes['brasil_vis_cumulative']

for var in themes['vars'].keys():
    for scale in themes['axis_legend']['scale'].keys():
        fig = vis_graphs.brasil_vis_cum(df_states, var,scale, themes)
        
        plot(fig, filename=f"../site/sv_br_vale/coronavirus/images/cumulative/brasil_por_estado_{var}.html", auto_open=False)


In [30]:
fig

## Diario

In [31]:
themes = yaml.load(open('../themes/custom_colorscales.yaml', 'r'), Loader=yaml.FullLoader)
themes = themes['brasil_vis_daily']

codes = ['BRASIL','SP']

for state in codes:
    mask = ((df_states['state']==state) & (df_states['confirmed']>0))
    fig = vis_graphs.unique_country(df_states[mask], themes)

    plot(fig, filename=f"../site/sv_br_vale/coronavirus/images/daily/{state}.html", auto_open=False)


In [32]:
fig

## MAPS

In [33]:
def _generate_and_upload_br_map(final, estados, map_config):
    #generate map
    cols = ['Município','Estado','Confirmados', 'Óbitos', 'Data do Boletim']
    
    print('Generate and save br map..')
    mymap =  vis_maps.get_map(final,'Confirmados', cols, estados)
    mymap.save(f'{map_config["path_save"]}{map_config["save_name"]}')

    print('Upload br map..')
    io.to_storage(bucket=map_config['bucket'],
              bucket_folder=map_config['bucket_folder'],
              file_name=map_config['save_name'],
              path_to_file=f'{map_config["path_save"]}{map_config["save_name"]}')
    
    os.remove(f'{map_config["path_save"]}{map_config["save_name"]}')

In [34]:
print('Start Br Map')
#minicipales data
municipios = pd.read_csv('../data/br_municipios_ibge.csv', dtype={'geocodigo':str})
df = brio.copy()

#load shape files;
ufs = gpd.read_file('../../mapas_brasil/estados-ibge/BRUFE250GC_SIR.shp')
maps = gpd.read_file('../../mapas_brasil/municipios-ibge/brasil-municipios.shp')

Start Br Map


In [35]:
from scripts import manipulation
from scripts import vis_maps

config = yaml.load(open('../configs/config.yaml', 'r'), Loader=yaml.FullLoader)

#prepare data for map
final, estados = manipulation.manipulate_for_br_maps(brio, municipios, maps, df_states.drop(['regiao'],1), ufs)

# _generate_and_upload_br_map(final, estados, config['br_map'])

## Vale

In [36]:
def _generate_and_upload_vale_map(vale, estados, map_config):
    #generate map
    cols = list(config['vale_map']['col_rename'].values())
    cols.remove('Fonte')
    
    print('Generate and save vale map..')
    mymap =  vis_maps.get_map_vale(vale,'Confirmados', cols, estados)
    mymap.save(f'{map_config["path_save"]}{map_config["save_name"]}')

    print('Map Done!')
#     io.to_storage(bucket=map_config['bucket'],
#               bucket_folder=map_config['bucket_folder'],
#               file_name=map_config['save_name'],
#               path_to_file=f'{map_config["path_save"]}{map_config["save_name"]}')
    
#     os.remove(f'{map_config["path_save"]}{map_config["save_name"]}')
#     return(mymap)

In [37]:

from scripts.io import read_sheets
from scripts import manipulation
#download and manipulate vale data
df = read_sheets('covid19_vale_do_paraiba_e_litoral_norte')
df = manipulation.manipulate_vale_data(df)

#load shape files
municipios_sp = gpd.read_file('../../brasil_geodata/maps/sp_municipios.json')
ufs = gpd.read_file('../../mapas_brasil/estados-ibge/BRUFE250GC_SIR.shp')


In [38]:
df.head()

,municipio,suspeitas,suspeitas_internados,confirmados,confirmados_internados,descartados,mortes_investigação,mortes,recuperados,ocupacao_enfermaria,ocupacao_uti,ultimo_boletim,ultima_atualizaçao,fonte,internados,nome_municipio
0,São José dos Campos,3803,137,4422,62,-,23,130,1578,79%,68%,14/07/2020,15/07/2020,https://www.instagram.com/prefeiturasjcamposoficial/?hl=en,199,São José dos Campos
1,São Sebastião,-,35,716,-,-,2,3,478,-,-,14/07/2020,15/07/2020,https://www.instagram.com/prefsaoseba/,35,São Sebastião
2,Jacareí,860,-,869,39,1371,16,48,632,-,-,14/07/2020,15/07/2020,https://www.instagram.com/prefeituradejacarei/,-,Jacareí
3,Taubaté,134,90,1000,-,1196,2,52,808,58%,70%,14/07/2020,15/07/2020,https://www.instagram.com/preftaubate/?hl=en,90,Taubaté
4,Caraguatatuba,168,-,526,-,1996,1,40,444,-,-,14/07/2020,15/07/2020,https://www.facebook.com/prefeituradecaraguatatuba,-,Caraguatatuba


In [39]:
## Manipulate data for generate map
config = yaml.load(open('../configs/config.yaml', 'r'), Loader=yaml.FullLoader)

vale, estados = manipulation.manipulate_for_vale_maps(df, municipios_sp, estados,config['vale_map'])

_generate_and_upload_vale_map(vale, estados, config['vale_map'])


Generate and save vale map..
Map Done!


In [40]:
#https://pypi.org/project/ftpretty/
from ftpretty import ftpretty
import ftplib
import yaml
import os
credentials = yaml.load(open('../../credentials/ftp_credentials.yaml', 'r'), Loader=yaml.FullLoader)


In [41]:
host = credentials['ftp']['host']
user = credentials['ftp']['user']
pw   = credentials['ftp']['pw']

f = ftpretty(host, user, pw)

### Upload sv site

In [42]:
%%time
local_folder  = '../site/sv_br_vale/coronavirus'
remote_folder = 'public_html/coronavirus'

f.upload_tree(local_folder, remote_folder)

CPU times: user 40.2 ms, sys: 62.2 ms, total: 102 ms
Wall time: 1min 9s


'public_html/coronavirus'

# Corona SP

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv', sep=';')

In [8]:
df['date'] = '2020-' + df['mes'].astype(str) + '-' + df['dia'].astype(str)

df['date'] = pd.to_datetime(df['date'])

In [20]:
df = df.sort_values(by=['munic','date'],ascending=True).drop(['dia','mes'],1)

In [25]:
mask = df['munic']=='taubate'
# df[mask]
